In [ ]:
import os
import requests
from openai import OpenAI
import gradio as gr
from dotenv import load_dotenv  
import google.generativeai as genai
from IPython.display import Markdown, display, update_display
load_dotenv(override=True)

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f'OpenAi api key exists and its starts with {openai_api_key[:3]}')
else:
    print("OpenAi api key doesn't exist")

if google_api_key:
    print('Google api key exists')
else:
    print("Google api key doesn't exist")

OPENAI_MODEL = "gpt-4o-mini"
GOOGLE_MODEL = "gemini-1.5-flash"

openai = OpenAI()

genai.configure()

In [ ]:
system_msg = """
You are CodeCopilot, an adaptive AI coding assistant that helps users solve problems in any programming language.
Always provide correct, runnable, and well-formatted code with clear explanations.
Adjust your style based on the user’s expertise: for beginners, break concepts down step by step with simple examples and commented code;
for advanced users, deliver concise, production-ready, optimized solutions with best practices and trade-off insights.
Ask clarifying questions when requirements are ambiguous, highlight pitfalls and edge cases,
and act as a collaborative pair programmer or mentor whose goal is to help users learn, build, and ship high-quality code efficiently.
"""


In [ ]:
def create_prompt(prompt, history):
    messages = [{"role": "system", "content": system_msg}]

    # history is a list of (user_msg, assistant_msg) tuples
    for user_msg, assistant_msg in history:
        if user_msg:
            messages.append({"role": "user", "content": user_msg})
        if assistant_msg:
            messages.append({"role": "assistant", "content": assistant_msg})

    # new user prompt
    messages.append({"role": "user", "content": prompt})
    return messages

In [ ]:
def openai_agent(prompt, history):
    openai.api_key = openai_api_key
    messages = create_prompt(prompt, history)
    response = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages,
        stream=True
    )
    sent_any = False
    for chunk in response:
        delta = chunk.choices[0].delta
        if delta and delta.content:
            sent_any = True
            yield delta.content
    if not sent_any:
        yield "(no response)"

In [ ]:
def gemini_agent(prompt, history):
    genai.configure(api_key=google_api_key)

    # reuse OpenAI-style messages
    messages = create_prompt(prompt, history)

    gemini_history = []
    for m in messages:
        # Gemini does NOT support system role
        if m["role"] == "system":
            continue
        gemini_history.append({
            "role": m["role"],
            "parts": [m["content"]]
        })
    prompt_with_system = f"{system_msg}\n\n{prompt}"
    model = genai.GenerativeModel(GOOGLE_MODEL)
    chat = model.start_chat(history=gemini_history)

    response = chat.send_message(prompt_with_system, stream=True)
    for chunk in response:
        if chunk and getattr(chunk, "text", None):
            yield chunk.text



In [ ]:
def chat_agent(prompt, history, modelType):
    if modelType == "OpenAI":
        for token in openai_agent(prompt, history):
            yield token
    else:
        for token in gemini_agent(prompt, history):
            yield token


In [ ]:
def chat_fn(prompt, history, model):
    assistant_response = ""
    for token in chat_agent(prompt, history, model):
        assistant_response += token
        yield assistant_response 

# -------------------------------------------------------------------
# UI
# -------------------------------------------------------------------
with gr.Blocks() as demo:
    model_choice = gr.Radio(["OpenAI", "Gemini"], value="OpenAI", label="Model")

    chat_ui = gr.ChatInterface(
        fn=chat_fn,
        additional_inputs=[model_choice],
        title="CodeCopilot",
        description="An adaptive AI coding assistant that helps developers build and ship high-quality code."
    )

demo.launch()